In [1]:
import re
import string
import json
from functools import lru_cache
import torch
import random
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import evaluation
from sentence_transformers import util
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses

In [2]:
class TokenSimilarity:

    def load_pretrained(self, from_pretrained: str = "indobenchmark/indobert-base-p1"):
        self.tokenizer = AutoTokenizer.from_pretrained(from_pretrained)
        self.model = AutoModel.from_pretrained(from_pretrained)

    def __cleaning(self, text: str):
        # Bersihkan tanda baca
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Bersihkan spasi ganda
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    def __process(self, first_token: str, second_token: str):
        inputs = self.tokenizer([first_token, second_token],
                                max_length=self.max_length,
                                truncation=self.truncation,
                                padding=self.padding,
                                return_tensors='pt')

        attention = inputs['attention_mask']

        outputs = self.model(**inputs)

        # Dapatkan bobot dari lapisan terakhir sebagai embeddings
        embeddings = outputs[0]

        # Tambahkan dimensi lebih lalu perluas tensor
        mask = attention.unsqueeze(-1).expand(embeddings.shape).float()

        masked_embeddings = embeddings * mask

        # MEAN POOLING UNTUK DIMENSI KE-2
        summed = masked_embeddings.sum(1)
        counts = torch.clamp(mask.sum(1), min=1e-9)
        mean_pooled = summed / counts

        # Kembalikan mean pooling sebagai array numpy
        return mean_pooled.detach().numpy()

    def predict(self, first_token: str, second_token: str,
                return_as_embeddings: bool = False, max_length: int = 16,
                truncation: bool = True, padding: str = "max_length"):
        self.max_length = max_length
        self.truncation = truncation
        self.padding = padding

        first_token = self.__cleaning(first_token)
        second_token = self.__cleaning(second_token)

        mean_pooled_arr = self.__process(first_token, second_token)
        if return_as_embeddings:
            return mean_pooled_arr

        # Hitung kemiripan
        similarity = cosine_similarity([mean_pooled_arr[0]], [mean_pooled_arr[1]])

        return similarity

model = TokenSimilarity()
model.load_pretrained('indobenchmark/indobert-base-p1')

In [3]:
# Load intent data
def load_intent_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

intent_data = load_intent_data('Dataset Chatbot.json')

In [4]:
# Training data
train_examples = [InputExample(texts=[pattern], label=intent['tag']) for intent in intent_data['intents'] for pattern in intent['patterns']]

# Model "use-moments" (base model)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [5]:
# Mapping antara tag dan label numerik
tag_to_label = {intent['tag']: idx for idx, intent in enumerate(intent_data['intents'])}
input_token = ""

def training_data(input_token):

    # Training data
    train_examples = []
    for intent in intent_data['intents']:
        for pattern in intent['patterns']:
                label = tag_to_label[intent['tag']]   # Mengonversi tag menjadi label numerik
                train_examples.append(InputExample(texts=[pattern, input_token], label=float(label))) # Mengonversi label ke float

    # Create a model
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    # Define a batch size
    batch_size = 32

    # Define a dataloader
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

    # Define a loss function
    train_loss = losses.CosineSimilarityLoss(model)

    # Training the model
    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2)

    return input_token

In [6]:
def match_intent(input_token, intent_data):
    input_embeddings = model.encode(input_token, convert_to_tensor=True)

    best_match = None
    best_similarity = -1  # Perhatikan bahwa similarity akan menjadi nilai -1 hingga 1.

    for intent in intent_data['intents']:
        for pattern in intent['patterns']:
            pattern_embedding = model.encode(pattern, convert_to_tensor=True)
            similarity = util.pytorch_cos_sim(input_embeddings, pattern_embedding)[0].item()

            if similarity > best_similarity:
                best_similarity = similarity
                best_match = (intent, pattern, similarity)

    return best_match

In [9]:
while True:
    try:
        input_user = input("You: ")
        if input_user in ("quit", "exit"):
            print("masuk break")
            break
        train = training_data(input_user)
        best_match = match_intent(train, intent_data)
        if best_match is not None:
                matched_intent, matched_pattern, similarity = best_match
                print(f"Input user: {input_user}")
                print(f"Token input cocok dengan intent: {matched_intent['tag']}")
                print(f"Pola terbaik: {matched_pattern}")
                print(f"Kemiripan: {similarity * 100:.2f}%")
                print("Bot : ",random.choice(matched_intent['response']))
        else:
                print("Harap berikan pertanyaan yang jelas.")
    except Exception as e:
        print("Terjadi kesalahan:", e)


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/25 [00:00<?, ?it/s]

Input user: apa itu aplikasi tanggap?
Token input cocok dengan intent: Aplikasi-Tanggap
Pola terbaik: Apa itu Aplikasi Tanggap?
Kemiripan: 100.00%
Bot :  Aplikasi Tanggap adalah sarana pengaduan masyarakat terhadap suatu masalah. Dengan aplikasi ini, Anda bisa melaporkan berbagai permasalahan yang dihadapi dan mendapatkan solusi atau tindak lanjut dari pihak berwenang.


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

...

Input user: Biaya UKT kuliah saya kenapa mahal sekali 
Token input cocok dengan intent: Suap
Pola terbaik: Saya melihat seseorang memberikan hadiah mahal kepada pejabat untuk memenangkan kontrak.
Kemiripan: 69.40%
Bot :  Terima kasih telah menghubungi kami. Suap adalah tindakan ilegal yang merugikan integritas sistem. Untuk melaporkan praktik suap, mohon sertakan informasi detail seperti nama, waktu, dan tempat kejadian, serta deskripsi spesifik dari tindakan yang Anda curigai. Informasi ini sangat penting untuk proses investigasi kami.


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/25 [00:00<?, ?it/s]

Input user: fasilitas sekolah jelek, banyak perabotan rusak
Token input cocok dengan intent: pengaduan-fasilitas-sekolah
Pola terbaik: Fasilitas olahraga di sekolah ini rusak
Kemiripan: 77.66%
Bot :  Terima kasih atas pertanyaan Anda mengenai pengaduan fasilitas di lingkungan sekolah. Kami mohon maaf atas pengalaman tidak menyenangkan yang Anda alami. Untuk mengajukan laporan pengaduan, silakan berikan deskripsi lengkap mengenai masalah yang Anda hadapi, termasuk alamat, foto, dan kondisi yang Anda alami di lingkungan sekolah. Terima kasih
masuk break
